In [1]:
# coding: utf-8
# Kaustuv Datta and Jayesh Mahaptra, July 2016 & Nikolaus Howe, May 2016
# pass in a string of events as a parameter, and it will parse and centre the events

import numpy as np
import sys
import ast
import h5py

In [2]:
# Given a list of distances and corresponding weights, calculate the weighted average
def findMidpoint(distance, energy):
    return np.average(distance, weights = energy)

In [3]:
# Given an array of interactions (ix,iy,iz,E,X,Y,Z), returns the weighted average (avex, avey, avez)
def findEventMidpoint(event):
    xave = findMidpoint(event[:,0], event[:,3]) 
    yave = findMidpoint(event[:,1], event[:,3]) 
    zave = findMidpoint(event[:,2], event[:,3]) 
    return (xave, yave, zave)

In [4]:
# Check if between bounds
def within(value, mymin, mymax):
    return (value >= mymin and value <= mymax)

In [15]:
# Given an event, get the 20x20x25 array of energies around its barycentre
def getECALArray(event):
    
    # Get the event midpoint
    midpoint = findEventMidpoint(event)
    print midpoint
    # Get the limit points for our grid
    xmin = midpoint[0] - 10
    xmax = midpoint[0] + 10
    ymin = midpoint[1] - 10
    ymax = midpoint[1] + 10
    
    # Create the empty array to put the energies in
    final_array = np.zeros((20, 20, 25))
    
    # Fill the array with energy values, if they exist
    #for element in event:
    #    if within(element[0], xmin, xmax) and within(element[1], ymin, ymax) and within(element[2], zmin, zmax):
    #        final_array[element[0], element[1], element[2]] = element[3]
  
    # Fill the array with energy values, if they exist
    for ix, iy, iz, E, x, y, z in event:
        if within(ix, xmin, xmax) and within(iy, ymin, ymax):
            final_array[ix-xmin,iy-ymin,iz] = E
    return final_array

In [16]:
# Given an event, get the 4x4x60 array of energies around its barycentre
def getHCALArray(event):
    
    # Get the event midpoint
    midpoint = findEventMidpoint(event)
    print midpoint
    # Get the limit points for our grid
    xmin = midpoint[0] - 2
    xmax = midpoint[0] + 2
    ymin = midpoint[1] - 2
    ymax = midpoint[1] + 2
    
    # Create the empty array to put the energies in
    final_array = np.zeros((4, 4, 60))
    
    # Fill the array with energy values, if they exist
    #for element in event:
    #    if within(element[0], xmin, xmax) and within(element[1], ymin, ymax) and within(element[2], zmin, zmax):
    #        final_array[element[0], element[1], element[2]] = element[3]
  
    # Fill the array with energy values, if they exist
    for ix, iy, iz, E, x, y, z in event:
        if within(ix, xmin, xmax) and within(iy, ymin, ymax):
            final_array[ix-xmin,iy-ymin,iz] = E
    return final_array

In [17]:
def convertFile(filename):
    with open(filename) as myfile:
        my_events_string = myfile.read().replace('\n', '')
    my_events = ast.literal_eval(my_events_string)
    event_list = []
    for event in my_events:
        event_list.append( np.array(event) )
    energy_array_list = []
    for event in event_list:
        energy_array_list.append(getEventArray(event))
    h5n = filename+'.h5'
    print "creating",h5n
    store = h5py.File(h5n)
    e = np.asarray( energy_array_list )
    store['images'] = e
    print e.shape
    store.close()

if __name__ == "__main__":
    import sys
    convertFile(sys.argv[1])

IOError: [Errno 2] No such file or directory: '-f'

In [18]:
fname = ("oneEVT.txt")

with open(fname) as file:
        events_string = file.read().replace('\n', '')
        my_events = (ast.literal_eval(events_string))

In [19]:
my_events = ast.literal_eval(events_string)
#print(len(my_events['ECAL']))
ECAL_list = []
for event in my_events['ECAL']:
    ECAL_list.append(np.array(event))
ECAL_array = np.array(ECAL_list)
ECAL_array_list = []
#for i in xrange(0, len(events_list)):
#    print (ECAL_array[i])
ECAL_array_list.append(getECALArray(ECAL_array))
ECAL_array_list[0].shape
#print(event_array_list)

(0.062626237807806961, -0.04926602204218572, 15.94869333676907)


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:24: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


(20, 20, 25)

In [23]:
#print(len(my_events['HCAL']))
HCAL_list = []
for event in my_events['HCAL']:
    HCAL_list.append(np.array(event))
HCAL_array = np.array(HCAL_list)
HCAL_array_list = []
#for i in xrange(0, len(events_list)):
#    print (ECAL_array[i])
HCAL_array_list.append(getHCALArray(HCAL_array))
HCAL_array_list[0].shape
#print(event_array_list)

(0.073720943616928492, -0.44460627665418018, 1.3249915174741627)


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:24: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


(4, 4, 60)

In [21]:
#collecting particle ID, energy of hit, and 3-vector of momentum
pdgID = my_events['pdgID']
if pdgID == 211 or pdgID == 111:
    pdgID = 0
if pdgID == 22:
    pdgID = 1
#collecting energy and momentum values from .txt file and converting from MeV to GeV
energy = my_events['E']
energy = energy/1000.
(px, py, pz) = (my_events['px'], my_events['py'], my_events['pz'])
(px, py, pz) = (px/1000., py/1000., pz/1000.)
p = [px, py, pz]
print (pdgID, energy, p)

(1, 98.99999999999997, [98.99999999999997, 0.0, 0.0])


In [29]:
f = h5py.File("EnergyScan_HCAL_ECAL.h5", "w")
target = np.zeros((1, 5))
target[:,0], target[:,1], target[:,2], target[:,3], target[:,4] = (pdgID, energy, px, py, pz)
f['target'] = target
f['ECAL'] = ECAL_array_list[0]
f['HCAL'] = HCAL_array_list[0]
f.close()

In [28]:
f.close()

In [31]:
 with open(fname) as myfile:
    my_events_string = myfile.read().replace('\n', '')
    my_events = ast.literal_eval(my_events_string)
    ECAL_list = []
    for event in my_events['ECAL']:
        ECAL_list.append(np.array(event))
    ECAL_array = np.array(ECAL_list)
    ECAL_array_list = []
    ECAL_array_list = []
    ECAL_array_list.append(getECALArray(ECAL_array))
    HCAL_list = []
    for event in my_events['HCAL']:
        HCAL_list.append(np.array(event))
    HCAL_array = np.array(HCAL_list)
    HCAL_array_list = []
    HCAL_array_list.append(getHCALArray(HCAL_array))
    
    #collecting particle ID, energy of hit, and 3-vector of momentum
    pdgID = my_events['pdgID']
    if pdgID == 211 or pdgID == 111:
        pdgID = 0
    if pdgID == 22:
        pdgID = 1
    energy = my_events['E']
    energy = energy/1000.
    (px, py, pz) = (my_events['px'], my_events['py'], my_events['pz'])
    (px, py, pz) = (px/1000., py/1000., pz/1000.)
    f = h5py.File("EnergyScan_HCAL_ECAL.h5", "w")
    target = np.zeros((1, 5))
    target[:,0], target[:,1], target[:,2], target[:,3], target[:,4] = (pdgID, energy, px, py, pz)
    f['target'] = target
    f['ECAL'] = ECAL_array_list[0]
    f['HCAL'] = HCAL_array_list[0]
    f.close()

(0.062626237807806961, -0.04926602204218572, 15.94869333676907)
(0.073720943616928492, -0.44460627665418018, 1.3249915174741627)


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:24: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
